In [1]:
import nltk
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from nltk.tokenize import word_tokenize
from collections import Counter
import re

In [2]:
document = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
Checkout this google sheet to see month by month time table of the course - https://docs.google.com/spreadsheets/d/16OoTax_A6ORAeCg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
"""

In [3]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
regex = re.compile(r'([^.])([.!?]+)([\s]*$)', re.MULTILINE)
document = regex.sub(r'\1\3', document)

In [5]:
tokenized_words = word_tokenize(document.lower())

In [6]:
vocab = {'<UNK>' : 0}

for word in Counter(tokenized_words).keys():
  vocab[word] = len(vocab)

In [7]:
input_sent = document.split('\n')

In [8]:
def text_to_index(vocab,sent):
  numerical_sent = []
  for token in sent:
    if token in vocab:
      numerical_sent.append(vocab[token])
    else:
      numerical_sent.append(vocab['<UNK>'])
  return numerical_sent

In [9]:
input_numerical_sent = []

for sent in input_sent:
  input_numerical_sent.append(text_to_index(vocab,word_tokenize(sent.lower())))

In [10]:
training_sequence = []
for sent in input_numerical_sent:
  for i in range(1,len(sent)):
    training_sequence.append(sent[:i+1])

In [11]:
len(training_sequence)

896

In [12]:
len_list = []
for seq in training_sequence:
  len_list.append(len(seq))

max(len_list)

61

In [13]:
padding_seq = []
for seq in training_sequence:
  padding_seq.append([0]*(max(len_list) - len(seq))+seq)

In [14]:
train_data = torch.tensor(padding_seq,dtype=torch.long)

In [15]:
X = train_data[:,:-1]
y = train_data[:,-1]

In [16]:
class trainData(Dataset):
  def __init__(self,X,y) -> None:
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, index):
    return self.X[index], self.y[index]

In [17]:
data = trainData(X,y)

In [18]:
dataloader = DataLoader(data,batch_size=32,shuffle=True,pin_memory=True)

In [19]:
class myLSTM(nn.Module):
  def __init__(self,vocab_size) -> None:
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,100)
    self.lstm = nn.LSTM(100,150,batch_first=True)
    self.linear = nn.Linear(150,vocab_size)

  def forward(self,X):
    embedded = self.embedding(X)
    intermediate_hidden_outs,(final_hidden_state,cell_state) = self.lstm(embedded)
    logits = self.linear(final_hidden_state.squeeze(0))

    return logits

In [27]:
model = myLSTM(len(vocab))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
learning_rate = 0.01
epochs = 20
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
criterior = nn.CrossEntropyLoss()

In [28]:
for epoch in range(epochs):
  total_loss = 0
  for X_batch,y_batch in dataloader:
    X_batch,y_batch = X_batch.to(device),y_batch.to(device)

    outs = model(X_batch)
    loss = criterior(outs,y_batch)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch + 1}, Loss: {total_loss:.4f}")

Epoch: 1, Loss: 146.7489
Epoch: 2, Loss: 101.6657
Epoch: 3, Loss: 60.4439
Epoch: 4, Loss: 34.3429
Epoch: 5, Loss: 19.9446
Epoch: 6, Loss: 13.0103
Epoch: 7, Loss: 9.3033
Epoch: 8, Loss: 6.9152
Epoch: 9, Loss: 5.5475
Epoch: 10, Loss: 5.2563
Epoch: 11, Loss: 4.6938
Epoch: 12, Loss: 4.4281
Epoch: 13, Loss: 4.3118
Epoch: 14, Loss: 3.9976
Epoch: 15, Loss: 3.8317
Epoch: 16, Loss: 4.0021
Epoch: 17, Loss: 3.9343
Epoch: 18, Loss: 3.7673
Epoch: 19, Loss: 3.8384
Epoch: 20, Loss: 3.7675


In [82]:
def predict(model,vocab,text):
  tokenized = word_tokenize(text.lower())
  indexed_text = text_to_index(vocab,tokenized)
  # Move the input tensor to the same device as the model
  padded_text =  torch.tensor([0] * (61 - len(indexed_text)) + indexed_text, dtype=torch.long).unsqueeze(0).to(next(model.parameters()).device)
  with torch.no_grad():
    logits = model(padded_text)
    _,index = torch.max(logits,dim=1)

  return text + " " + list(vocab.keys())[index]

In [84]:
predict(model,vocab,'Can I do this course')

'Can I do this course if'

In [88]:
import time

num_tokens = 10
input_text = "You can watch the videos"

for i in range(num_tokens):
  output_text = predict(model, vocab, input_text)
  print(output_text)
  input_text = output_text
  time.sleep(0.5)

You can watch the videos till
You can watch the videos till your
You can watch the videos till your subscription
You can watch the videos till your subscription is
You can watch the videos till your subscription is valid
You can watch the videos till your subscription is valid .
You can watch the videos till your subscription is valid . suppose
You can watch the videos till your subscription is valid . suppose you
You can watch the videos till your subscription is valid . suppose you have
You can watch the videos till your subscription is valid . suppose you have purchased


In [67]:
dataloader1 = DataLoader(data, batch_size=32, shuffle=False)

In [68]:
def calculate_accuracy(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No need to compute gradients
        for batch_x, batch_y in dataloader1:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            # Get model predictions
            outputs = model(batch_x)

            # Get the predicted word indices
            _, predicted = torch.max(outputs, dim=1)

            # Compare with actual labels
            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)

    accuracy = correct / total * 100
    return accuracy

# Compute accuracy
accuracy = calculate_accuracy(model, dataloader, device)
print(f"Model Accuracy: {accuracy:.2f}%")


Model Accuracy: 95.42%
